**RESNIK-SIMILARITY**

In [0]:
import string
nltk.download('wordnet_ic')
from nltk.corpus import wordnet as wn
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
from nltk.corpus import wordnet_ic
brown_ic = wordnet_ic.ic('ic-brown.dat')
ps=PorterStemmer()
table = str.maketrans('', '', string.punctuation)
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package wordnet_ic to /root/nltk_data...
[nltk_data]   Package wordnet_ic is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

Reading the train and test data

In [0]:
data = pd.read_csv('/content/drive/My Drive/train.csv')
testdata = pd.read_csv('/content/drive/My Drive/test.csv')
real_word=[]
nonreal_word=[]

Making the real and non-real word set from train data

In [0]:
for i in range(5000):
    if data['target'][i]==0:
        tokens = nltk.word_tokenize(data['text'][i])
        tokens = [w.lower() for w in tokens]
        table = str.maketrans('', '', string.punctuation)
        stripped = [w.translate(table) for w in tokens]
        words = [word for word in stripped if word.isalpha()]
        stop_words = set(stopwords.words('english'))
        words = [w for w in words if not w in stop_words]
        lmt = [wordnet_lemmatizer.lemmatize(word) for word in words]
        lm = [ps.stem(word) for word in lmt]
        for l in lm:
            if l not in nonreal_word:
                if wn.synsets(l,pos='n'):
                    nonreal_word.append(l)
    else:
        tokens = nltk.word_tokenize(data['text'][i])
        tokens = [w.lower() for w in tokens]
        table = str.maketrans('', '', string.punctuation)
        stripped = [w.translate(table) for w in tokens]
        words = [word for word in stripped if word.isalpha()]
        stop_words = set(stopwords.words('english'))
        words = [w for w in words if not w in stop_words]
        lmt = [wordnet_lemmatizer.lemmatize(word) for word in words]
        lm = [ps.stem(word) for word in lmt]
        for l in lm:
            if l not in real_word:
                if wn.synsets(l,pos='n'):
                    real_word.append(l)

Finding the label for each text according to its similarity with real and non-real word set

In [0]:
original =[]
computed=[]

for lk in range(5001,len(data['target'])):
    word=[]
    tokens = nltk.word_tokenize(data['text'][lk])
    tokens = [w.lower() for w in tokens]
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    words = [word for word in stripped if word.isalpha()]
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    lmt = [wordnet_lemmatizer.lemmatize(word) for word in words]
    lm = [ps.stem(word) for word in lmt]
    simi1=[]
    simi2=[]
    for l in lm:
        if l not in word:
            if wn.synsets(l,pos='n'):
                word.append(l)
   
    if len(word)==0:
        continue
    for i in word:
        try:
            s = wn.synset(i+'.n.01')
        except:
            s = None
            continue
        for k in real_word:
            try:
                s2 = wn.synset(k+'.n.01')
            except:
                s2=None
                continue
            
            sim = s.res_similarity(s2,brown_ic)
            simi1.append(sim)
  #calculate resnik similarity between the synonym or the sense of each word of text with sense of each real word
    for i in real_word:
        try:
            s = wn.synset(i+'.n.01')
        except:
            s = None
            continue
        for k in word:
            try:
                s2 = wn.synset(k+'.n.01')
            except:
                s2=None
                continue
            sim = s.res_similarity(s2,brown_ic)
            simi2.append(sim)
    #find the average of the similarities of two words which have the maximum similarity
    similarity1 = (max(simi1)+max(simi2))/2
    simi1=[]
    simi2=[]
    for i in word:
        try:
            s = wn.synset(i+'.n.01')
        except:
            s = None
            continue
        for k in nonreal_word:
            try:
                s2 = wn.synset(k+'.n.01')
            except:
                s2=None
                continue
            sim = s.res_similarity(s2,brown_ic)
            simi1.append(sim)
    #calculate resnik similarity between the synonym or the sense of each word of text with sense of each non-real word
    for i in nonreal_word:
        try:
            s = wn.synset(i+'.n.01')
        except:
            s = None
            continue
        for k in word:
            try:
                s2 = wn.synset(k+'.n.01')
            except:
                s2=None
                continue
            sim = s.res_similarity(s2,brown_ic)
            simi2.append(sim)
    #find the average of the similarities of two words which have the maximum similarity
    similarity2 = (max(simi1)+max(simi2))/2
    # Assigning label which has higher similarity
    if(similarity1>similarity2):
        original.append(data['target'][lk])
        computed.append(1)
    else:
        original.append(data['target'][lk])
        computed.append(0)

Accuracy of Resnik similarity

In [0]:
print("**********************************************************************")
print("Accuracy of Resnik Similarity is as follow:- ")   
print(accuracy_score(original,computed))

**********************************************************************
Accuracy of Resnik Similarity is as follow:- 
0.5538759689922481
